In [40]:
import pandas
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Concatenate

2024-02-26 12:57:19.330588: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 12:57:19.639140: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 12:57:19.639263: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 12:57:19.699968: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-26 12:57:19.928730: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 12:57:19.936922: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
data = pandas.read_csv("CrimeData.csv")

In [29]:
# totalData = []
# for index, row in data.iterrows():
#     if row['DISTRICT'] == "TOTAL":
#         # print(row)
#         totalData.append(row)
# print(totalData)
# totalData = pandas.DataFrame(totalData)
# totalData.to_csv("totalCrime.csv")

[STATE/UT                                               ANDHRA PRADESH
DISTRICT                                                        TOTAL
YEAR                                                             2001
MURDER                                                           2602
ATTEMPT TO MURDER                                                1555
CULPABLE HOMICIDE NOT AMOUNTING TO MURDER                         136
RAPE                                                              871
CUSTODIAL RAPE                                                      0
OTHER RAPE                                                        871
KIDNAPPING & ABDUCTION                                           1182
KIDNAPPING AND ABDUCTION OF WOMEN AND GIRLS                       765
KIDNAPPING AND ABDUCTION OF OTHERS                                417
DACOITY                                                           214
PREPARATION AND ASSEMBLY FOR DACOITY                                9
ROBBERY            

In [35]:
crime_data = pandas.read_csv('totalCrime.csv')

In [36]:
crime_data

,state,year,total_crime
0,ANDHRA PRADESH,2001,130089
1,ARUNACHAL PRADESH,2001,2342
2,ASSAM,2001,36877
3,BIHAR,2001,88432
4,CHHATTISGARH,2001,38460
...,...,...,...
403,CHANDIGARH,2012,3606
404,D & N HAVELI,2012,318
405,DAMAN & DIU,2012,239
406,LAKSHADWEEP,2012,60


In [38]:
# Preprocess the data
# Assuming you want to predict total crime based on the year and city
# You may need to perform additional preprocessing based on your specific dataset
crime_data['year'] = pandas.to_datetime(crime_data['year']) 

In [42]:
# Encode city names
encoder = LabelEncoder()
crime_data['state_encoded'] = encoder.fit_transform(crime_data['state'])

In [43]:
# Normalize the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(crime_data[['total_crime', 'state_encoded']])

In [44]:
# Prepare data for LSTM
# Assume we want to use the past 5 years' data to predict the next year's total crime
look_back = 5
X, y = [], []
for i in range(len(scaled_data) - look_back):
    X.append(scaled_data[i:(i + look_back), :])
    y.append(scaled_data[i + look_back, 0])
X, y = np.array(X), np.array(y)

In [45]:
# Build LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(LSTM(units=50))
model.add(Dense(units=1))

2024-02-26 12:58:58.302735: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-26 12:58:58.303384: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [46]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [127]:
model.fit(X, y, epochs=100, batch_size=32)

Epoch 1/100
13/13 [==============================] - 0s 10ms/step - loss: 0.0014
Epoch 2/100
13/13 [==============================] - 0s 10ms/step - loss: 0.0013
Epoch 3/100
13/13 [==============================] - 0s 11ms/step - loss: 0.0012
Epoch 4/100
13/13 [==============================] - 0s 10ms/step - loss: 0.0013
Epoch 5/100
13/13 [==============================] - 0s 10ms/step - loss: 0.0011
Epoch 6/100
13/13 [==============================] - 0s 10ms/step - loss: 0.0011
Epoch 7/100
13/13 [==============================] - 0s 10ms/step - loss: 0.0012
Epoch 8/100
13/13 [==============================] - 0s 13ms/step - loss: 0.0011
Epoch 9/100
13/13 [==============================] - 0s 12ms/step - loss: 0.0012
Epoch 10/100
13/13 [==============================] - 0s 12ms/step - loss: 0.0011
Epoch 11/100
13/13 [==============================] - 0s 13ms/step - loss: 0.0011
Epoch 12/100
13/13 [==============================] - 0s 16ms/step - loss: 0.0013
Epoch 13/100
13/13 [=====

In [128]:
# Prediction
city_name = 'MAHARASHTRA'

# Filter data for the specific city
city_data = crime_data[crime_data['state'] == city_name]

In [129]:
# Encode the city name
encoded_city = encoder.transform([city_name])[0]

In [130]:
# Select the most recent 5 years' data
recent_data = city_data.tail(look_back)

In [131]:
# Prepare input data
input_data = recent_data[['total_crime', 'state_encoded']]
input_data['state_encoded'] = encoded_city  # Add encoded city column
input_data = np.array(input_data)


/tmp/ipykernel_26748/3142184061.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data['state_encoded'] = encoded_city  # Add encoded city column


In [132]:
# Reshape input data to match the input shape of the LSTM model
input_data = np.reshape(input_data, (1, look_back, input_data.shape[1]))

In [133]:
# Make prediction
predicted_scaled_crime = model.predict(input_data)

1/1 [==============================] - 0s 26ms/step


In [134]:
predicted_crime = scaler.inverse_transform(np.array([[predicted_scaled_crime[0, 0], encoded_city]]))

In [135]:
print(f"Predicted total crime for {city_name} for the next year: {int(predicted_crime[0, 0])}")


Predicted total crime for MAHARASHTRA for the next year: 381148
